# European Option Pricing with QuantLib

In this notebook we illustrate simple option pricing with QuantLib.

The notebook is structured as follows:

  1. Setup of market data and model

  2. Pricing Vanilla options with CRR model and Black-Scholes model.

  3. Pricing Barrier options with Black-Scholes model.

In [ ]:
import QuantLib as ql
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## Market Data and Model Setup

We set up simple constant term structures for risk-free rate and volatility. As inputs we use the parameters from our Black-Scholes example

In [ ]:
r = -0.0058
sigma = 0.48
S0 = 1.0

### YieldTermStructure

In [ ]:
discountYts  = ql.FlatForward(0, ql.NullCalendar(), r, ql.Actual365Fixed())
discountYtsH = ql.YieldTermStructureHandle(discountYts)

### BlackVolTermStructure

In [ ]:
volTs  = ql.BlackConstantVol(0, ql.NullCalendar(), sigma, ql.Actual365Fixed())
volTsH = ql.BlackVolTermStructureHandle(volTs)

### BlackScholesProcess

In [ ]:
asset = ql.QuoteHandle(ql.SimpleQuote(S0))
process = ql.BlackScholesProcess(asset, discountYtsH, volTsH)

## Vanilla Option Pricing

### Day Count Convention, Model Times and Option Exercise

In QuantLib instrument properties relating to times are specified in actual dates. In order to translate dates into model times $T$, *day count conventions* are used.

A day count convention is a metric on calender dates. A general calculation formula for a time difference or year fraction $YF$ is
$$
  YF = \frac{\text{\# days between dates}}{\text{\# days per year}}.
$$
There are various market conventions for how to calculate the numerator and denominator. We use the *Actual/365 (Fixed)* day count convention which divides the difference in actual calender days by $365$.

As base date for valuation we take today.

In [ ]:
today = ql.Settings.getEvaluationDate(ql.Settings.instance())
print(today)

In our earlier example we assumed a time to expiry of $T=1.4$. We interpret this duration using the Actual/365 (Fixed) day count convention.

In [ ]:
T = 1.4

numberOfDays = int(np.round(T * 365))
numberOfDays

Now we can calculate the option expiry date.

In [ ]:
expiryDate = today + numberOfDays
print(expiryDate)

With the expiry date we can now specify a European exercise event.

In [ ]:
exercise = ql.EuropeanExercise(expiryDate)

### Payoffs

We use the same strikes as in our Black-Scholes example.

In [ ]:
putStrikes  = [ 0.60,   0.70,   0.80,   0.90,   1.00   ]
callStrikes = [ 1.00,   1.10,   1.20,   1.30,   1.40   ]

Payoffs are separate objects in QuantLib. 

In [ ]:
putPayoffs  = [ ql.PlainVanillaPayoff(ql.Option.Put,K)  for K in putStrikes  ]
callPayoffs = [ ql.PlainVanillaPayoff(ql.Option.Call,K) for K in callStrikes ]

### Instruments

Exercise and payoff are combined to set up a Vanilla option.

In [ ]:
puts  = [ ql.EuropeanOption(p,exercise) for p in putPayoffs  ]
calls = [ ql.EuropeanOption(p,exercise) for p in callPayoffs ]

### Black-Scholes Pricing

In order to price an instrument in QuantLib we need a *PricingEngine*. A PricingEngine specifies the model and valuation method and links it to the option instrument.

In [ ]:
bsEngine = ql.AnalyticEuropeanEngine(process)

for i in puts + calls:
    i.setPricingEngine(bsEngine)

Now, we can ask the instrument for its price and sensitivities.

In [ ]:
putTableBS = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta', 'Rho', 'Vega'))
for K, i in zip(putStrikes, puts):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta(), i.rho(), i.vega() )
    putTableBS = putTableBS.append({ c : v for c,v in zip(putTableBS.columns,res) }  , ignore_index=True)
print('Puts:')
print(putTableBS)

callTableBS = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta', 'Rho', 'Vega'))
for K, i in zip(callStrikes, calls):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta(), i.rho(), i.vega() )
    callTableBS = callTableBS.append({ c : v for c,v in zip(callTableBS.columns,res) }  , ignore_index=True)
print('Calls:')
print(callTableBS)


### CRR Model Pricing

In order to apply a CRR model we just need to setup a corresponding pricing engine.

The critical parameter in the CRR model is the number of time steps $N$. This is an additional parameter to the PricingEngine.

In [ ]:
N = 3

crrEngine = ql.BinomialCRRVanillaEngine(process, N)

Now, we can link the new engine to the instruments and repeat pricing. Note that QuantLib's CRR engine does not implement Rho and Vega.

In [ ]:
for i in puts + calls:
    i.setPricingEngine(crrEngine)

In [ ]:
putTableCRR = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(putStrikes, puts):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    putTableCRR = putTableCRR.append({ c : v for c,v in zip(putTableCRR.columns,res) }  , ignore_index=True)
print('Puts:')
print(putTableCRR)

callTableCRR = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(callStrikes, calls):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    callTableCRR = callTableCRR.append({ c : v for c,v in zip(callTableCRR.columns,res) }  , ignore_index=True)
print('Calls:')
print(callTableCRR)

Finally, we can plot and compare the BS and CRR model results.

In [ ]:
def plotResults(resString):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=putStrikes,  y=putTableBS[resString],   name='put  BS',  line=dict(color='royalblue', dash=None) ))
    fig.add_trace(go.Scatter(x=callStrikes, y=callTableBS[resString],  name='call BS',  line=dict(color='firebrick', dash=None) ))
    fig.add_trace(go.Scatter(x=putStrikes,  y=putTableCRR[resString],  name='put  CRR', line=dict(color='royalblue', dash='dash') ))
    fig.add_trace(go.Scatter(x=callStrikes, y=callTableCRR[resString], name='call CRR', line=dict(color='firebrick', dash='dash') ))
    fig.update_layout(
        title='Black-Scholes and CRR Model %s, T=%.2f' % (resString,T),
        xaxis_title="Strike K",
        yaxis_title=resString,
        width=600, height=400, autosize=False,
        #margin=dict(l=65, r=50, b=65, t=90),
    )
    fig.show()
    return

plotResults('Price')
plotResults('Delta')
plotResults('Gamma')
plotResults('Theta')

## Barrier Option Pricing

Barrier options are another instrument type in QuantLib.

For a barrier option we need to specify the barrier type and barrier level. We choose to use down-and-out put (DOP) up-and-in calls (UIC). As put barrier level we set $B=0.5$ and as call barrier level we set $B=2.0$. 

In [ ]:
lowBarrier = 0.5
upBarrier  = 2.0
rebate     = 0.0 # we do not model rebate

dop  = [ ql.BarrierOption(ql.Barrier.DownOut, lowBarrier, rebate, p, exercise) for p in putPayoffs  ]
uic  = [ ql.BarrierOption(ql.Barrier.UpIn,    upBarrier,  rebate, p, exercise) for p in callPayoffs ]

### Black-Scholes Pricing

We use an AnalyticBarrierEngine to specify the model and pricing method.

In [ ]:
bsBarrierEngine = ql.AnalyticBarrierEngine(process)

for i in dop + uic:
    i.setPricingEngine(bsBarrierEngine)

In [ ]:
barrierPutTableBS = pd.DataFrame(columns=('Strike', 'Price'))
for K, i in zip(putStrikes, dop):
    res = ( K, i.NPV() )
    barrierPutTableBS = barrierPutTableBS.append({ c : v for c,v in zip(barrierPutTableBS.columns,res) }  , ignore_index=True)
print('Barrier Puts:')
print(barrierPutTableBS)

barrierCallTableBS = pd.DataFrame(columns=('Strike', 'Price'))
for K, i in zip(callStrikes, uic):
    res = ( K, i.NPV() )
    barrierCallTableBS = barrierCallTableBS.append({ c : v for c,v in zip(barrierCallTableBS.columns,res) }  , ignore_index=True)
print('Barrier Calls:')
print(barrierCallTableBS)

### CRR Model Pricing

We use a BinomialCRRBarrierEngine for barrier option pricing on a binomial tree.

In [ ]:
N = 3
crrBarrierEngine = ql.BinomialCRRBarrierEngine(process, N)

for i in dop + uic:
    i.setPricingEngine(crrBarrierEngine)

In [ ]:
barrierPutTableCRR = pd.DataFrame(columns=('Strike', 'Price'))
for K, i in zip(putStrikes, dop):
    res = ( K, i.NPV() )
    barrierPutTableCRR = barrierPutTableCRR.append({ c : v for c,v in zip(barrierPutTableCRR.columns,res) }  , ignore_index=True)
print('Barrier Puts:')
print(barrierPutTableCRR)

barrierCallTableCRR = pd.DataFrame(columns=('Strike', 'Price'))
for K, i in zip(callStrikes, uic):
    res = ( K, i.NPV() )
    barrierCallTableCRR = barrierCallTableCRR.append({ c : v for c,v in zip(barrierCallTableCRR.columns,res) }  , ignore_index=True)
print('Barrier Calls:')
print(barrierCallTableCRR)

Finally, we can compare CRR prices to BS prices.

In [ ]:
resString = 'Price'
fig = go.Figure()
fig.add_trace(go.Scatter(x=putStrikes,  y=barrierPutTableBS[resString],   name='DOP BS',  line=dict(color='royalblue', dash=None) ))
fig.add_trace(go.Scatter(x=callStrikes, y=barrierCallTableBS[resString],  name='UIC BS',  line=dict(color='firebrick', dash=None) ))
fig.add_trace(go.Scatter(x=putStrikes,  y=barrierPutTableCRR[resString],  name='DOP CRR', line=dict(color='royalblue', dash='dash') ))
fig.add_trace(go.Scatter(x=callStrikes, y=barrierCallTableCRR[resString], name='UIC CRR', line=dict(color='firebrick', dash='dash') ))
fig.update_layout(
    title='Black-Scholes and CRR Model Barrier %s, T=%.2f, N=%d' % (resString,T,N),
    xaxis_title="Strike K",
    yaxis_title=resString,
    width=600, height=400, autosize=False,
    #margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()
